## Download and prep the data

In [1]:
# Dataset
# In this homework, we will use the lead scoring dataset Bank Marketing dataset.

# run once
# !wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv

# In this dataset our desired target for classification task will be converted variable - has the client signed up to the platform or not.

# Data preparation
# Check if the missing values are presented in the features.
# If there are missing values:
# -- For categorical features, replace them with 'NA'
# -- For numerical features, replace with with 0.0

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

df = pd.read_csv('course_lead_scoring.csv')
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [3]:
# we will select numerical and categorical columns from this list:
df.columns

Index(['lead_source', 'industry', 'number_of_courses_viewed', 'annual_income',
       'employment_status', 'location', 'interaction_count', 'lead_score',
       'converted'],
      dtype='object')

In [4]:
# lets see how pandas typed our columns - object means categorical column
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [5]:
numerical = ['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']
numerical 

# list of numerical features 
# ['number_of_courses_viewed',
#  'annual_income',
#  'interaction_count',
#  'lead_score']

# converted will be our label - we will train our model and predict it

['number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score']

In [6]:
categorical = ['lead_source', 'industry', 'employment_status', 'location']
categorical

# list of categorical features
# ['lead_source', 'industry', 'employment_status', 'location']

['lead_source', 'industry', 'employment_status', 'location']

In [7]:
# how many missing values in our dataframe per column
df.isna().sum()


lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [8]:
# Check if the missing values are presented in the features.

for col in df[categorical]:
    print(f"{col}:")
    print(df[col].unique())
    print("-" * 40)

lead_source:
['paid_ads' 'social_media' 'events' 'referral' 'organic_search' nan]
----------------------------------------
industry:
[nan 'retail' 'healthcare' 'education' 'manufacturing' 'technology'
 'other' 'finance']
----------------------------------------
employment_status:
['unemployed' 'employed' nan 'self_employed' 'student']
----------------------------------------
location:
['south_america' 'australia' 'europe' 'africa' 'middle_east' nan
 'north_america' 'asia']
----------------------------------------


In [9]:
# If there are missing values:
# -- For categorical features, replace them with 'NA'

df[categorical] = df[categorical].fillna('NA') # to fill NaN in-place
for col in df[categorical]:
    print(f"{col}:")
    print(df[col].unique())
    print("-" * 40)
                       

lead_source:
['paid_ads' 'social_media' 'events' 'referral' 'organic_search' 'NA']
----------------------------------------
industry:
['NA' 'retail' 'healthcare' 'education' 'manufacturing' 'technology'
 'other' 'finance']
----------------------------------------
employment_status:
['unemployed' 'employed' 'NA' 'self_employed' 'student']
----------------------------------------
location:
['south_america' 'australia' 'europe' 'africa' 'middle_east' 'NA'
 'north_america' 'asia']
----------------------------------------


In [10]:
# If there are missing values:
# -- For numerical features, replace with with 0.0

df[numerical] = df[numerical].fillna(0.0) # to fill NaN in-place
# CHECK - how many missing values in our dataframe per column NOW:
df.isna().sum() # all good!

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

## Question 1

In [11]:
# Question 1
# What is the most frequent observation (mode) for the column industry?

# NA
# technology
# healthcare
# retail -- retail           203

df['industry'].value_counts()

industry
retail           203
finance          200
other            198
healthcare       187
education        187
technology       179
manufacturing    174
NA               134
Name: count, dtype: int64

## Question 2

In [12]:
# Question 2
# Create the correlation matrix for the numerical features of your dataset. 
# In a correlation matrix, you compute the correlation coefficient between every pair of features.

# What are the two features that have the biggest correlation?

# -- interaction_count and lead_score = 0.009888
# -- number_of_courses_viewed and lead_score = -0.004879
# -- number_of_courses_viewed and interaction_count = -0.023565
# -- annual_income and interaction_count = 0.027036 - this is the biggest correlation
# Only consider the pairs above when answering this question.

corr_matrix = df[numerical].corr()
corr_matrix

,number_of_courses_viewed,annual_income,interaction_count,lead_score
number_of_courses_viewed,1.000000,0.009770,-0.023565,-0.004879
annual_income,0.009770,1.000000,0.027036,0.015610
interaction_count,-0.023565,0.027036,1.000000,0.009888
lead_score,-0.004879,0.015610,0.009888,1.000000


In [13]:
# Split the data
# Split your data in train/val/test sets with 60%/20%/20% distribution.
# Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.

from sklearn.model_selection import train_test_split

In [14]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [15]:
# lets check how SKL split our dataset
len(df_train), len(df_val), len(df_test) # (876, 293, 293)

(876, 293, 293)

In [16]:
# we need to reset index in each split for simplicity
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [17]:
# ... and remove label we predict - so our model will not accidentally look at it...
# Make sure that the target value converted is not in your dataframe.

y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

del df_train['converted']
del df_val['converted']
del df_test['converted']

In [18]:
# CHECKING
df_train.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score
0,paid_ads,retail,0,58472.0,student,middle_east,5,0.03
1,organic_search,manufacturing,3,71738.0,student,middle_east,6,0.77
2,paid_ads,technology,3,81973.0,employed,north_america,2,0.59
3,NA,technology,1,74956.0,employed,europe,3,0.34
4,organic_search,retail,3,59335.0,student,australia,1,0.98


In [19]:
df_val.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score
0,paid_ads,healthcare,3,52220.0,unemployed,europe,1,0.07
1,organic_search,technology,3,59656.0,unemployed,middle_east,4,0.65
2,events,manufacturing,0,57134.0,self_employed,north_america,4,0.13
3,events,other,0,0.0,NA,asia,0,0.03
4,referral,retail,1,54103.0,unemployed,south_america,3,0.16


In [20]:
df_test.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score
0,social_media,manufacturing,2,56070.0,self_employed,middle_east,2,0.23
1,NA,other,1,78409.0,NA,australia,4,0.79
2,referral,manufacturing,2,66206.0,employed,australia,3,0.30
3,events,retail,0,0.0,self_employed,north_america,2,0.98
4,organic_search,retail,6,62832.0,unemployed,NA,4,1.00


In [21]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NA,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NA,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


## Question 3

In [22]:
# Question 3
# Calculate the mutual information score between converted and other categorical variables in the dataset. Use the training set only.
# Round the scores to 2 decimals using round(score, 2).
# Which of these variables has the biggest mutual information score?

# -- industry -- 0.01
# -- location -- 0.0
# -- lead_source -- 0.04 -- lead sourcce has the biggest mutual info score
# -- employment_status -- 0.01

In [23]:
# mutual information - how much we can explain variations of one variable by observing another
# it is kinda similar to correlation coefficient - but for categorical features ...
# in practice SKL has it:

from sklearn.metrics import mutual_info_score 

In [24]:
# Calculate the mutual information score between converted and other categorical variables in the dataset. Use the training set only.

round(mutual_info_score(df_train.industry, y_train), 2) 
# 0.01

0.01

In [25]:
round(mutual_info_score(df_train.location, y_train), 2) 
# 0.0

0.0

In [26]:
round(mutual_info_score(df_train.lead_source, y_train), 2) 
# 0.04

0.04

In [27]:
round(mutual_info_score(df_train.employment_status, y_train), 2) 
# 0.01

0.01

## Question 4

In [28]:
# Question 4
# Now let's train a logistic regression.
# Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
# Fit the model on the training dataset.
# To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
# model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
# Calculate the accuracy on the validation dataset and round it to 2 decimal digits.
# What accuracy did you get?

# 0.64
# 0.74 -- np.float64(0.7)
# 0.84
# 0.94

from sklearn.feature_extraction import DictVectorizer

In [29]:
# fit and transform train and val datasets by 1-hot encoding
dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict) # WE USE ONLY TRANSFORM IN VAL DATASET - DO NOT FIT THERE!!!
# fit_transform is the same as dv.fit and dv.transform

In [30]:
# CHECK 1
categorical, numerical # to check ourselves if we have our col names still correct

(['lead_source', 'industry', 'employment_status', 'location'],
 ['number_of_courses_viewed',
  'annual_income',
  'interaction_count',
  'lead_score'])

In [31]:
# CHECK 2
dv.get_feature_names_out()

array(['annual_income', 'employment_status=NA',
       'employment_status=employed', 'employment_status=self_employed',
       'employment_status=student', 'employment_status=unemployed',
       'industry=NA', 'industry=education', 'industry=finance',
       'industry=healthcare', 'industry=manufacturing', 'industry=other',
       'industry=retail', 'industry=technology', 'interaction_count',
       'lead_score', 'lead_source=NA', 'lead_source=events',
       'lead_source=organic_search', 'lead_source=paid_ads',
       'lead_source=referral', 'lead_source=social_media', 'location=NA',
       'location=africa', 'location=asia', 'location=australia',
       'location=europe', 'location=middle_east',
       'location=north_america', 'location=south_america',
       'number_of_courses_viewed'], dtype=object)

In [32]:
# CHECK 3
list(X_val[0]) # 4 numerical cols and many 1-hot encoded categorical...

[np.float64(52220.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(1.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(1.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(1.0),
 np.float64(0.07),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(1.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(1.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(3.0)]

In [33]:
# CHECK 4
X_train.shape, X_val.shape # ((876, 31), (293, 31))

((876, 31), (293, 31))

In [34]:
from sklearn.linear_model import LogisticRegression

# lets train it
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)
model

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [35]:
# Calculate the accuracy on the validation dataset and round it to 2 decimal digits.
# What accuracy did you get?

# 0.64
# 0.74 -- np.float64(0.7)
# 0.84
# 0.94

# get test predictions using model.predict
# y_pred = model.predict_proba(X_val)[:, 1]
y_pred = model.predict(X_val)
# y_pred


In [36]:
round((y_pred == y_val).mean(), 2)
# np.float64(0.7)

np.float64(0.7)

## Question 5

In [37]:
# Question 5
# Let's find the least useful feature using the feature elimination technique.
# Train a model using the same features and parameters as in Q4 (without rounding).
# Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
# For each feature, calculate the difference between the original accuracy and the accuracy without the feature.
# Which of following feature has the smallest difference?

# -- 'industry' = Feature excluded: industry precision change:  0.0
# -- 'employment_status'
# -- 'lead_score'
# Note: The difference doesn't have to be positive.
categorical

['lead_source', 'industry', 'employment_status', 'location']

In [38]:
# Train a model using the same features and parameters as in Q4 (without rounding)
original = (y_pred == y_val).mean() 
original
# np.float64(0.6996587030716723)

np.float64(0.6996587030716723)

In [39]:
# Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
# For each feature, calculate the difference between the original accuracy and the accuracy without the feature.
# Which of following feature has the smallest difference?

def exclude_feature(categorical=categorical):
    """
    takes list of categorical features, excludes one by one and train the model with remaining features
    """
    for c in categorical:
        new_feature_list = [f for f in categorical if f!=c]

        # fit and transform train and val datasets by 1-hot encoding
        dv = DictVectorizer(sparse=False)
        
        train_dict = df_train[new_feature_list + numerical].to_dict(orient='records')
        X_train = dv.fit_transform(train_dict)
        
        val_dict = df_val[new_feature_list + numerical].to_dict(orient='records')
        X_val = dv.transform(val_dict) # WE USE ONLY TRANSFORM IN VAL DATASET - DO NOT FIT THERE!!!
        
        # CHECK 2
        # print()
        # print(dv.get_feature_names_out())
        # print('-'*40)
        model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_val)
        res = (y_pred == y_val).mean()
        precision_change = original - res
        print('Feature excluded:', c, 'precision change: ', precision_change)

        

In [40]:
exclude_feature(categorical)

Feature excluded: lead_source precision change:  -0.0034129692832765013
Feature excluded: industry precision change:  0.0
Feature excluded: employment_status precision change:  0.0034129692832763903
Feature excluded: location precision change:  -0.010238907849829393


## Question 6

In [41]:
# Question 6
# Now let's train a regularized logistic regression.
# Let's try the following values of the parameter C: [0.01, 0.1, 1, 10, 100].
# Train models using all the features as in Q4.
# Calculate the accuracy on the validation dataset and round it to 3 decimal digits.
# Which of these C leads to the best accuracy on the validation set?

# 0.01
# 0.1
# 1
# 10
# 100
# Note: If there are multiple options, select the smallest C.



In [42]:
def regularized_regression(c):
    # fit and transform train and val datasets by 1-hot encoding
    dv = DictVectorizer(sparse=False)
    
    train_dict = df_train[categorical + numerical].to_dict(orient='records')
    X_train = dv.fit_transform(train_dict)
    
    val_dict = df_val[categorical + numerical].to_dict(orient='records')
    X_val = dv.transform(val_dict) # WE USE ONLY TRANSFORM IN VAL DATASET - DO NOT FIT THERE!!!
    
    model = LogisticRegression(solver='liblinear', C=c, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_val)
    res = round((y_pred == y_val).mean(), 3)
    # res = (y_pred == y_val).mean()
    print('Hyperparameter c value: ', c, ' accuracy: ', res)

        

In [43]:
for c in [0.01, 0.1, 1, 10, 100]:
    regularized_regression(c)

# It all looks the same - maybe model predicting 1 major class only regardless of hyperparameter C:
# Hyperparameter c value:  0.01  accuracy:  0.7
# Hyperparameter c value:  0.1  accuracy:  0.7
# Hyperparameter c value:  1  accuracy:  0.7
# Hyperparameter c value:  10  accuracy:  0.7
# Hyperparameter c value:  100  accuracy:  0.7

Hyperparameter c value:  0.01  accuracy:  0.7
Hyperparameter c value:  0.1  accuracy:  0.7
Hyperparameter c value:  1  accuracy:  0.7
Hyperparameter c value:  10  accuracy:  0.7
Hyperparameter c value:  100  accuracy:  0.7


## Improvements

In [44]:
# 1. Class Imbalance: Check if one class dominates (~70%)
#    - If yes, the model may default to predicting majority class
#    - Try class_weight='balanced' in LogisticRegression

# 2. Weak Features: All correlations < 0.03 is very weak
#    - Consider feature engineering
#    - Look for non-linear relationships
   
# 3. Try Different Metrics: Accuracy may hide poor performance
#    - Use precision, recall, F1-score
#    - Look at ROC-AUC score
   
# 4. Next Steps:
#    - Try model.predict_proba() to see confidence scores
#    - Use different threshold than 0.5
#    - Try different models (Random Forest, XGBoost)



In [45]:
#    - Try class_weight='balanced' in LogisticRegression

def regularized_regression(c):
    # fit and transform train and val datasets by 1-hot encoding
    dv = DictVectorizer(sparse=False)
    
    train_dict = df_train[categorical + numerical].to_dict(orient='records')
    X_train = dv.fit_transform(train_dict)
    
    val_dict = df_val[categorical + numerical].to_dict(orient='records')
    X_val = dv.transform(val_dict) # WE USE ONLY TRANSFORM IN VAL DATASET - DO NOT FIT THERE!!!
    
    model = LogisticRegression(
        solver='liblinear', 
        C=c, 
        max_iter=1000, 
        random_state=42,
        class_weight='balanced' # NB - added this DESPITE homework recommendations!!!
    )
    model.fit(X_train, y_train)

    y_pred = model.predict(X_val)
    res = round((y_pred == y_val).mean(), 3)
    # res = (y_pred == y_val).mean()
    print('Hyperparameter c value: ', c, ' accuracy: ', res)

        

In [46]:
for c in [0.001, 0.01, 0.1, 0.5, 1, 5, 10, 100, 1000]:
    # Extended range for C DESPITE of [0.01, 0.1, 1, 10, 100] recommended:
    regularized_regression(c)

# a bit better but still mostly the same - no reaction for hyperparameter tuning:
# Hyperparameter c value:  0.01  accuracy:  0.795
# Hyperparameter c value:  0.1  accuracy:  0.846
# Hyperparameter c value:  1  accuracy:  0.846
# Hyperparameter c value:  10  accuracy:  0.846
# Hyperparameter c value:  100  accuracy:  0.846

# using extended C range DESPITE of homework recommendations
# [0.001, 0.01, 0.1, 0.5, 1, 5, 10, 100, 1000]

# Hyperparameter c value:  0.001  accuracy:  0.747
# Hyperparameter c value:  0.01  accuracy:  0.795
# Hyperparameter c value:  0.1  accuracy:  0.846
# Hyperparameter c value:  0.5  accuracy:  0.86 == this is the winner so far...
# Hyperparameter c value:  1  accuracy:  0.846
# Hyperparameter c value:  5  accuracy:  0.843
# Hyperparameter c value:  10  accuracy:  0.846
# Hyperparameter c value:  100  accuracy:  0.846
# Hyperparameter c value:  1000  accuracy:  0.846

Hyperparameter c value:  0.001  accuracy:  0.747
Hyperparameter c value:  0.01  accuracy:  0.795
Hyperparameter c value:  0.1  accuracy:  0.846
Hyperparameter c value:  0.5  accuracy:  0.86
Hyperparameter c value:  1  accuracy:  0.846
Hyperparameter c value:  5  accuracy:  0.843
Hyperparameter c value:  10  accuracy:  0.846
Hyperparameter c value:  100  accuracy:  0.846
Hyperparameter c value:  1000  accuracy:  0.846


In [47]:
# Class Imbalance: Check if one class dominates (~70%)
#    - If yes, the model may default to predicting majority class

# Check class balance
df['converted'].value_counts(normalize=True)
# If the dataset is roughly 70% one class, the model is likely just predicting the majority class.
# converted
# 1    0.619015
# 0    0.380985
# Name: proportion, dtype: float64
# RESULT = class balance is OK-ish...

converted
1    0.619015
0    0.380985
Name: proportion, dtype: float64

## Hyperparameter tuning results

In [48]:
# so after some tweaking we can reach 0.86 accuracy with
    # model = LogisticRegression(
    #     solver='liblinear', 
    #     C=0.5, # NB - added this extra
    #     max_iter=1000, 
    #     random_state=42,
    #     class_weight='balanced' # NB - added this extra
    # )